In [71]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras import layers
from tensorflow.keras import losses
import re
import string

In [72]:
# url = 'stack_overflow_16k.tar.gz'
# dataset = tf.keras.utils.get_file(
#     "stack_overflow_16k.tar.gz", url, untar=True, cache_dir= '.',
#     cache_subdir = '')
dataset_dir = 'stackoverflow_dataset'


In [73]:
os.listdir(dataset_dir)

['README.md', 'test', 'train']

In [74]:
train_dir = os.path.join('stackoverflow_dataset', 'train')
test_dir = os.path.join('stackoverflow_dataset', 'test')

In [121]:
class_names = os.listdir(train_dir)
print(class_names)

['csharp', 'java', 'javascript', 'python']


In [76]:
sample_file = os.path.join(train_dir, 'csharp/0.txt')
with open(sample_file) as f:
    print(f.read())

"how to pause loop while time is ticking i have made a timer where i can set time i want to wait and then do something..so this is my short timer func:..private void sleep(int interval, action action).{.    system.windows.forms.timer mytimer = new system.windows.forms.timer();.    mytimer.interval = interval; //interval is in ms   .    mytimer.start();.    mytimer.tick += (s, e) =&gt;.    {.        action();.        mytimer.stop();.    };.}...and im using this timer in loop:..foreach (string word in words).{.   sleep(5000, ()=&gt; myaction());                                           .}...without loop timer is great but in loop it wont work because loop wont stop and wait for those 5secs. it do all stuff imediately and starts timer again and again too fast...so what im trying to find out is how to make my loop wait until time runs out and myaction() is executed. im working on forms application so all threadin sleeps wont work here. also i tried all other timers but they used too much 

In [77]:
batch_size = 32
seed = 42
raw_train_ds =tf.keras.utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    seed=seed,
    validation_split = 0.2,
    subset = 'training'
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [78]:
class_names = raw_train_ds.class_names

In [79]:
print(class_names)

['csharp', 'java', 'javascript', 'python']


In [80]:
raw_val_ds =tf.keras.utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    seed=seed,
    validation_split = 0.2,
    subset = 'validation'
)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [81]:
raw_test_ds =tf.keras.utils.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size,
    
)

Found 8000 files belonging to 4 classes.


In [82]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [83]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [94]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [95]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, axis=-1)
    return vectorize_layer(text),label

In [96]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [97]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [98]:
embedding_dim = 16

In [106]:
model = tf.keras.Sequential([
    layers.Embedding(max_features, embedding_dim),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_6      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [107]:
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics = ['Accuracy']
              )

In [110]:
epochs = 50
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - Accuracy: 0.8785 - loss: 0.4022 - val_Accuracy: 0.7931 - val_loss: 0.5374
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - Accuracy: 0.8847 - loss: 0.3873 - val_Accuracy: 0.7956 - val_loss: 0.5276
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - Accuracy: 0.8926 - loss: 0.3757 - val_Accuracy: 0.7987 - val_loss: 0.5234
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - Accuracy: 0.8905 - loss: 0.3711 - val_Accuracy: 0.8006 - val_loss: 0.5207
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - Accuracy: 0.8929 - loss: 0.3622 - val_Accuracy: 0.7969 - val_loss: 0.5232
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - Accuracy: 0.8933 - loss: 0.3532 - val_Accuracy: 0.7962 - val_loss: 0.5182
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - Accuracy: 0.9026 - loss: 0.3437 - val_Accuracy: 0.8031 - val_loss: 0.5128
Epoch 8/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - Accuracy: 0.8974 - loss: 0.3363 - val_Accuracy: 0.

In [111]:
loss, accuracy = model.evaluate(test_ds)

250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 53ms/step - Accuracy: 0.7903 - loss: 0.6259


In [113]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('softmax')
])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

# Test it with `raw_test_ds`, which yields raw strings
metrics = export_model.evaluate(raw_test_ds, return_dict=True)
print(metrics)

250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - Accuracy: 0.0000e+00 - accuracy: 0.7927 - loss: 0.0000e+00
{'Accuracy': 0.0, 'accuracy': 0.7885000109672546, 'loss': 0.0}


In [144]:
examples = tf.constant([
 "how do i display the original date using system datetime blank i used the the code below to fine people if they brought the book after it was due. but when i calculate the date of the duedate is changing to 1/1/0001?..i need help keeping the original duedate displayed...private short bookloan;..private void btncal_click(object sender, eventargs e).{.    system.datetime datedueback = new system.datetime();..    long daysoverdue = 0;.    decimal finerate = default(decimal);.    decimal fine = default(decimal);..    txtdatedue.text = datedueback.tostring();..    daysoverdue= microsoft.visualbasic.dateandtime.datediff(""d"", datedueback, txtdatereturned.text);..    switch (numericupdown1.decimalplaces) {.        case 0:.            bookloan = 1;.            finerate = 2;.            break;.        case 1:.            bookloan = 3;.            finerate = 3;.            break;.        case 2:.            bookloan = 10;.            finerate = 5;.            break;.        case 3:.            bookloan = 30;.            finerate = 35;..            break;.    }..    if (daysoverdue &lt;= 0) {.        daysoverdue = 0;.    }..    if (daysoverdue &gt; 0) {.        fine = finerate * daysoverdue;.    } else {.        fine = 0;.    }..    txtdateoverdue.text = daysoverdue + "" days overdue"";.    txtreturnfine.text = ""fine of "" + strings.format(fine, ""currency"") + "" to pay"";.}"



])

predict= export_model.predict(examples)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [145]:
class_id = np.argmax(predict, axis=1)
pred_class= class_names[class_id[0]]

In [146]:
print(pred_class)

csharp
